The Task
Your task is two-fold:

I. Clean the files and combine them into one final DataFrame.

This dataframe should have the following columns:
- Hero (Just the name of the Hero)
- Publisher
- Gender
- Eye color
- Race
- Hair color
- Height (numeric)
- Skin color
- Alignment
- Weight (numeric)
- Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
    - Agility
    - Flight
    - Superspeed
    - etc.
    
Hint: There is a space in "100 kg" or "52.5 cm"



In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

## Importing the OS and JSON Modules
import os,json

## Filter ALL warnings
import warnings
warnings.filterwarnings('ignore')

# Super Hero CSV

In [2]:
# CSV File
sup_hero = pd.read_csv('Data/superhero_info.csv')
sup_hero.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [3]:
sup_hero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [4]:
# Duplicate values
print(f'There are {sup_hero.duplicated().sum()} duplicate rows.')

There are 0 duplicate rows.


Separate the Hero and Publisher

In [5]:
sup_hero[['Hero', 'Publisher']] = sup_hero['Hero|Publisher'].str.split('|', expand = True)

sup_hero.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [6]:
sup_hero = sup_hero.drop(columns=['Hero|Publisher'])
sup_hero.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics
3,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics
4,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics


Separate the Height and Weight

In [7]:
## examining a single value from the coordinates col
measurements = sup_hero.loc[0,"Measurements"]
print(type(measurements))
measurements


<class 'str'>


"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [8]:
measurements = measurements.replace("'",'"')
measurements

'{"Height": "203.0 cm", "Weight": "441.0 kg"}'

In [9]:
## now we can use json.loads
fixed_measurements = json.loads(measurements)
print(type(fixed_measurements))
fixed_measurements


<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [10]:
## use .str.replace to replace all single quotes
sup_hero['Measurements'] = sup_hero['Measurements'].str.replace("'",'"')
## Apply the json.loads to the full column
sup_hero['Measurements'] = sup_hero['Measurements'].apply(json.loads)
sup_hero['Measurements'].head()


0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: Measurements, dtype: object

In [11]:
## check a single value after transformation
test_measurements = sup_hero.loc[0, 'Measurements']
print(type(test_measurements))
test_measurements


<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [12]:
ht_wt = sup_hero['Measurements'].apply(pd.Series)
ht_wt

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [13]:
# concat ht_wt with original dataframe
sup_hero = pd.concat((sup_hero, ht_wt), axis = 1)
sup_hero.head(2)


,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


Remove the old Column

In [14]:
sup_hero = sup_hero.drop(columns=['Measurements'])
sup_hero.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg


In [15]:
sup_hero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Gender      463 non-null    object
 1   Race        463 non-null    object
 2   Alignment   463 non-null    object
 3   Hair color  463 non-null    object
 4   Eye color   463 non-null    object
 5   Skin color  463 non-null    object
 6   Hero        463 non-null    object
 7   Publisher   463 non-null    object
 8   Height      463 non-null    object
 9   Weight      463 non-null    object
dtypes: object(10)
memory usage: 36.3+ KB


Clean Columns convert data type

In [16]:
sup_hero['Weight'] = sup_hero['Weight'].str.replace(' kg', '')
sup_hero['Weight'].head(2)

0    441.0
1     65.0
Name: Weight, dtype: object

In [45]:
sup_hero['Weight'] = sup_hero['Weight'].astype(float)

In [18]:
sup_hero['Height'] = sup_hero['Height'].str.replace(' cm', '')
sup_hero['Height'].head(2)

0    203.0
1    191.0
Name: Height, dtype: object

In [44]:
sup_hero['Height'] = sup_hero['Height'].astype(float)

In [20]:
sup_hero.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0


In [46]:
sup_hero.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Gender      463 non-null    object 
 1   Race        463 non-null    object 
 2   Alignment   463 non-null    object 
 3   Hair color  463 non-null    object 
 4   Eye color   463 non-null    object 
 5   Skin color  463 non-null    object 
 6   Hero        463 non-null    object 
 7   Publisher   463 non-null    object 
 8   Height      463 non-null    float64
 9   Weight      463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


# Super Powers CSV

In [21]:
# CSV File
sup_powers = pd.read_csv('Data/superhero_powers.csv')
sup_powers.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [22]:
sup_powers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [23]:
# Duplicate values
print(f'There are {sup_powers.duplicated().sum()} duplicate rows.')

There are 0 duplicate rows.


In [24]:
sup_powers['powers_split'] = sup_powers['Powers'].str.split(',')


In [25]:
exploded = sup_powers.explode('powers_split')


In [26]:
cols_to_make = exploded['powers_split'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [27]:
for col in cols_to_make:
    sup_powers[col] = sup_powers['Powers'].str.contains(col)
sup_powers.head()

,hero_names,Powers,powers_split,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre...",False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [28]:
# Drop the unnecessary columns
sup_powers = sup_powers.drop(columns=['Powers', 'powers_split'])
sup_powers.head()

,hero_names,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,Camouflage,Self-Sustenance,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,True,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,True,False,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Combine Dataframe

In [29]:
## merging with different column names
hero_df = pd.merge(sup_hero, sup_powers, left_on='Hero', right_on='hero_names')

hero_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,...,False,False,False,False,False,False,False,False,False,False


II. Use your combined DataFrame to answer the following questions.

1. Compare the average weight of super powers who have Super Speed to those who do not.


In [37]:
# With super speed
speed = hero_df['Super Speed'] == True

speed_df = hero_df[speed]

speed_df



,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
5,Male,Human,good,Blond,blue,Unknown,Adam Strange,DC Comics,185.0,88.0,...,False,False,False,False,False,False,False,False,False,False
8,Male,Unknown,bad,White,blue,Unknown,Air-Walker,Marvel Comics,188.0,108.0,...,False,False,False,False,False,False,False,False,False,False
9,Male,Cyborg,bad,Black,brown,Unknown,Ajax,Marvel Comics,193.0,90.0,...,False,False,False,False,False,False,False,False,False,False
10,Male,Unknown,good,Blond,blue,Unknown,Alan Scott,DC Comics,180.0,90.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,Female,Demi-God,good,Blond,blue,Unknown,Wonder Girl,DC Comics,165.0,51.0,...,False,False,False,False,False,False,False,False,False,False
454,Male,Unknown,good,Black,red,Unknown,Wonder Man,Marvel Comics,188.0,171.0,...,False,False,False,False,False,False,False,False,False,False
455,Female,Amazon,good,Black,blue,Unknown,Wonder Woman,DC Comics,183.0,74.0,...,False,False,False,False,False,False,False,False,False,False
460,Male,Yoda's species,good,White,brown,green,Yoda,George Lucas,66.0,17.0,...,False,False,False,False,False,False,False,False,False,False


In [48]:
# weights
speed_df['Weight'].mean

<bound method NDFrame._add_numeric_operations.<locals>.mean of 3      441.0
5       88.0
8      108.0
9       90.0
10      90.0
       ...  
453     51.0
454    171.0
455     74.0
460     17.0
462     81.0
Name: Weight, Length: 198, dtype: object>

In [41]:
no_speed_df = hero_df[~speed]

265

In [42]:
no_speed_df['Weight'].mean

<bound method NDFrame._add_numeric_operations.<locals>.mean of 0      441.0
1       65.0
2       90.0
4      122.0
6       81.0
       ...  
456     50.0
457     61.0
458     83.0
459     52.0
461     57.0
Name: Weight, Length: 265, dtype: object>

2. What is the average height of heroes for each publisher?

In [32]:
height = hero_df.groupby('Publisher')['Height'].mean()
height.sort_values()

TypeError: Could not convert 185.0185.0180.0257.0183.0178.0185.0198.0178.0168.0183.0203.0170.0165.0168.0188.0178.0178.0173.0188.0191.0191.0165.0170.0185.0188.0183.0196.0198.0170.0170.0193.0193.0175.0175.0163.0170.0175.0183.0198.0267.0183.0185.0193.0188.0175.0244.0185.0168.0193.0188.0180.0183.0183.0157.062.5198.0178.0188.0188.0188.0170.0185.0175.0185.0180.0180.0170.0185.0196.0173.0244.0234.064.0180.0188.0165.0155.0191.0229.0201.0180.0175.0196.0185.0185.0183.0178.0183.0193.0178.0178.0157.0168.0185.0168.0168.0180.0180.0188.0193.0165.0178.0180.0183.0165.0185.0185.0178.0183.0165.0137.0168.0183.0173.0201.0175.0279.0188.0193.0165.0170.0201.0183.0170.0180.0165.0191.0188.0183.0168.0178.0175.0173.0165.0183.0170.0185.0 to numeric